# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

# Objectives

The objective of this project is to:
- Prepare a report for a bank's loan division by analyze a borrower's risk of defaulting.
- Apply Data Preprocessing to a real-life analytical case study.

<hr>

 # Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#open_the_data">Open the data file and have a look at the general information</a></li>
        <li><a href="#data_preprocessing">Data preprocessing</a></li>
        <li><a href="#answer_questions">Answer these questions</a></li>
        <li><a href="#general_conclusion">General conclusion</a></li>
        <li><a href="#project_readiness_checklist">Project Readiness Checklist</a></li>
    </ol>
</div>
<br>
<hr>

<div id="open_the_data">
    <h2>Open the data file and have a look at the general information.</h2> 
</div>

In [32]:
# import pandas for data preprocessing and manipulation
import pandas as pd

# read in the data
try:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')
except:
    df = pd.read_csv('credit_scoring_eng_eng.csv')

In [33]:
# use the head() method to view the first 10 rows
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [34]:
# check the datatype with info() method
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [35]:
# check the structure of our dataset
df.shape

(21525, 12)

From the initial look at the data, we can see that `days_employed` and `total_income` has some missing data or 'NaNs'. We can see that the columns with missing values is of the float datatype. From the shape of the dataset, there are 21525 rows and 12 columns. 

### Conclusion

By initially looking at the dataset and general information about the data, we can get a picture of the structure of the data, the datatypes, number of rows and columns in the data. Now we proceed to determine the number of missing values and see what significance their absence is absence is for the dataset.  

<div id="data_preprocessing">
    <h2>Data preprocessing</h2> 
</div>

### Processing missing values

In [36]:
# print the number of missing values per column
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [37]:
# function to calculate the percentage of missing values
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [38]:
missing_values_table(df)

Your selected dataframe has 12 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
days_employed,2174,10.1
total_income,2174,10.1


From the above processing of missing values, we can see that about 10% of data on how long the customer has been working <em>"days_employed"</em> and monthly income <em>"total_income"</em> is missing. We need to decide whether the missing data is low enough for rows with missing data to be dropped or replacing the missing data with the mean or median is necessary. We proceed to determine whether the data has significant outliers by determining summary statistics for the rows with missing values.

In [41]:
# use describe() to check for outliers
print(df['days_employed'].describe())
print()
print(df['total_income'].describe())

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64


From the `describe()` method, for the column `days_employed`, the mean is 63046 and the median is -1203.36. This means that the dataset contains significant outliers. For the column `total_income`, the mean is 26787 and the median is 23202. We could likely replace missing values in this column with the mean or median. Checking through the missing values, we observe that the missing values exhibit a 'systematic' pattern. For each rows with missing data, the `days_employed` and `total_income` values are all missing. This means the data is "Missing not at random (MNAR)". That means those who refused to give information about the days employed will likely not give information about their total income. In this case, we have to drop those rows.

In [50]:
# count of missing data by gender
print('Count of missing days employed by gender')
print(df[df['days_employed'].isnull()]['gender'].value_counts())

print()
print('Count of total income missing by gender')
print(df[df['total_income'].isnull()]['gender'].value_counts())

Count of missing days employed by gender
F    1484
M     690
Name: gender, dtype: int64

Count of total income missing by gender
F    1484
M     690
Name: gender, dtype: int64


The count of missing data by gender shows that the data is missing not at random. i.e. same number of both gender are missing in the `days_employed` and `total_income` columns.

In [70]:
# count of missing data by income type
print('Count of missing days employed by income type')
print(df[df['days_employed'].isnull()]['income_type'].value_counts())

Count of missing days employed by income type
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64


In [75]:
# dropping rows with missing values
credit_score_df = df.dropna().reset_index(drop=True)

In [76]:
# check to see if missing data still exist
missing_values_table(credit_score_df)

Your selected dataframe has 12 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [78]:
# check new dataframe info
credit_score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null float64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null float64
purpose             19351 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


### Conclusion

From the Data Preprocessing section, we computed the number of missing values per column and the percentage of missing values per column. We can see that 10% of rows are missing `days_employed` and `total_income`. Since the type of data with missing values are quantitative, and the data are "Missing not at random", we proceed to drop those rows with missing values. We drop missing rows because 10% of missing values would not really skew our analysis.  

### Data type replacement

After dropping rows, we still observe that the `days_employed` and `total_income` are of float datatype. We need to replace data type and convert from 'float' to 'int'.

In [92]:
# change float to int in days_employed column
credit_score_df['days_employed'] = credit_score_df['days_employed'].astype('int')

# change float to int in total_income column
credit_score_df['total_income'] = credit_score_df['total_income'].astype('int')

credit_score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19351 entries, 0 to 19350
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 1.8+ MB


### Conclusion

We are able to change the `days_employed` and `total_income` from floating point value to integer datatype by using the `astype()` method and the `('int')` argument.

### Processing duplicates

### Conclusion

### Categorizing Data

### Conclusion

<div id="answer_questions">
    <h2>Answer these questions</h2> 
</div>

- Is there a relation between having kids and repaying a loan on time?

### Conclusion

- Is there a relation between marital status and repaying a loan on time?

### Conclusion

- Is there a relation between income level and repaying a loan on time?

### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

### Conclusion

<div id="general_conclusion">
    <h2>General conclusion</h2> 
</div>

<div id="project_readiness_checklist">
    <h2>Project Readiness Checklist</h2> 
</div>

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [ ]  file examined;
- [ ]  missing values defined;
- [ ]  missing values are filled;
- [ ]  an explanation of which missing value types were detected;
- [ ]  explanation for the possible causes of missing values;
- [ ]  an explanation of how the blanks are filled;
- [ ]  replaced the real data type with an integer;
- [ ]  an explanation of which method is used to change the data type and why;
- [ ]  duplicates deleted;
- [ ]  an explanation of which method is used to find and remove duplicates;
- [ ]  description of the possible reasons for the appearance of duplicates in the data;
- [ ]  data is categorized;
- [ ]  an explanation of the principle of data categorization;
- [ ]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [ ]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [ ]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [ ]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [ ]  conclusions are present on each stage;
- [ ]  a general conclusion is made.